In [2]:
import os
from collections import defaultdict
import nltk
from baseline_utils import process_baseline
from nltk.corpus import stopwords
import pprint
from Levenshtein import distance
import re
import json
import requests
import io
import numpy as np

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/adaamko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
dictionary = defaultdict(list)
with open("/home/adaamko/tools/wikt2dict/dat/wiktionary/Hungarian/dictionary", "r+") as f:
    for line in f:
        line = line.strip().split("\t")
        if line[0] == "en":
            dictionary[line[1].lower()].append(line[3].lower())

In [5]:
from collections import defaultdict
dictionary_hok = defaultdict(list)
count=0
with open("/home/adaamko/data/hokoto", errors="replace") as f:
    for line in f:
        line = line.strip().split("@")
        count+=1
        dictionary_hok[line[0]].append(line[2])

In [6]:
old_char = ["a1", "e1", "u1", "i1", "o1", "A1", "E1", "U1", "I1", "O1", "o2", "u2", "O2", "U2", "o3", "u3", "O3", "U3", "_"]
new_char = ["á", "é", "ú", "í", "ó", "Á", "É", "Ú", "Í", "Ó", "ö", "ü", "Ö", "Ü", "ő", "ű", "Ő", "Ű", " "]

In [7]:
for i in dictionary_hok:
    for j in range(len(dictionary_hok[i])):
        for k in range(len(old_char)):
            dictionary_hok[i][j] = dictionary_hok[i][j].replace(old_char[k], new_char[k])

In [8]:
for word_hok in dictionary_hok:
    words = dictionary_hok[word_hok]
    dictionary[word_hok] += words

In [9]:
sentences = process_baseline("/home/adaamko/data/1984.sen-aligned.np-aligned.gold")
len(sentences)

6567

In [10]:
len(stop_sentences)

NameError: name 'stop_sentences' is not defined

In [11]:
import spacy
import hu_core_ud_lg

In [12]:
nlp_hu = hu_core_ud_lg.load()
nlp_en = spacy.load("en_core_web_sm")

In [13]:
import emmorphpy.emmorphpy as emmorph

In [40]:
lem_hu = emmorph.EmMorphPy()

In [ ]:
stop_sentences = []

for i,sentence in enumerate(sentences):
    stop_count_en = 0
    stop_count_hu = 0
    
    if i % 100 == 0:
        print(i)
    
    for s in sentence["en_sen"]:
        if type(s) == tuple:
            doc = nlp_en(' '.join(s[1]))
            nps = []
            for ent in doc:
                if not ent.is_stop:
                    nps.append(ent.lemma_)
            if not nps:
                stop_count_en += 1
                
    for s in sentence["hu_sen"]:
        if type(s) == tuple:
            doc = nlp_hu(' '.join(s[1]))
            nps = []
            for ent in doc:
                if not ent.is_stop:
                    nps.append(ent.lemma_)
            if not nps:
                stop_count_hu += 1
    if stop_count_en < 4 and stop_count_hu < 4:
        stop_sentences.append(sentence)

In [16]:
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [17]:
src_path = '/home/adaamko/data/DMR/wiki.multi.en.vec'
tgt_path = '/home/adaamko/data/DMR/wiki.multi.hu.vec'
nmax = 250000  # maximum number of word embeddings to load

src_embeddings, src_id2word, src_word2id = load_vec(src_path, nmax)
tgt_embeddings, tgt_id2word, tgt_word2id = load_vec(tgt_path, nmax)

In [18]:
src_word2id = {v: k for k, v in src_id2word.items()}
tgt_word2id = {v: k for k, v in tgt_id2word.items()}

def get_distance(src_word, tgt_word, src_emb, tgt_emb):
    src_word_emb = src_emb[src_word2id[src_word]]
    tgt_word_emb = tgt_emb[tgt_word2id[tgt_word]]
    score = (tgt_word_emb / np.linalg.norm(tgt_word_emb)).dot(src_word_emb / np.linalg.norm(src_word_emb))
    return score

In [19]:
src_word = "instrument"
tgt_word = "készülék"

get_distance(src_word, tgt_word, src_embeddings, tgt_embeddings)

0.3969420479092985

In [20]:
def filter_nps(nps, language):
    lemmas = []
    if language == "hu":
        #words = [word for word in nps if word.isupper() == False]
        #words = [word.lower() for word in words if word.lower() not in stopwords.words('hungarian')]
        words = [word for word in nps if word not in stopwords.words('hungarian')]
        if not words:
            words = nps
        for np in words:
            try:
                if len(lem_hu.stem(np)) > 0:
                    lemma = lem_hu.stem(np)[0][0]
                    lemmas.append(lemma)
                    lemmas.append(np)
                else:
                    lemmas.append(np)
            except (IndexError, NameError) as e:
                lemmas.append(np)
        
    if language == "en":
        #words = [word.lower() for word in nps if word.lower() not in stopwords.words('english')]
        words = [word for word in nps if word not in stopwords.words('english')]
        if not words:
            words = nps
        for np in words:
            if len(nlp_en(np)) > 0:
                lemma = nlp_en(np)[0].lemma_
            else:
                lemma = np
            if lemma == "-PRON-":
                lemmas.append(np.lower())
            else:
                #lemmas.append(lemma.lower())
                lemmas.append(lemma)
    return lemmas

In [21]:
def return_morph(word):
    s = lem_hu.analyze(word)
    ret_list = []
    for i in s:
        i = i.split("=")
        morpheme = i[1]
        morpheme = morpheme.split("+")
        for m in morpheme:
            mor = m.split("[")[0].strip()
            if len(mor) > 2:
                ret_list.append(m.split("[")[0].strip())
    return ret_list

In [22]:
from collections import Counter
def count_vowels(word):
    c = {v:word.count(v) for v in 'aeuioáéúüűíóöő'}
    count = sum(c.values())
    return count

In [23]:
def get_embedding_score(en_np, hu_np):
    max_score = 0
    ancestors_word_en = []
    for word in en_np:
        w = word.strip("-").lower()
        if w in en_ancestors:
            for anc in en_ancestors[w]:
                if anc not in en_ancestor_top:
                    ancestors_word_en.append(anc)
        ancestors_word_en.append(w)
    
    ancestors_word_hu = []
    for hu_word in hu_np:
        if hu_word in hu_ancestors:
            for anc in hu_ancestors[hu_word]:
                if anc not in hu_ancestor_top:
                    ancestors_word_hu.append(anc)
        ancestors_word_hu.append(hu_word)
        
    for word in ancestors_word_en:
        for hu_word in ancestors_word_hu:
            try:
                distance = get_distance(word, hu_word, src_embeddings, tgt_embeddings)
            except KeyError:
                distance = 0
            if distance > max_score:
                max_score = distance
    return max_score

In [43]:
def compute_scores(sen):
    en_nps = {}
    hu_nps = {}
    for s in sen['en_sen']:
        if type(s) == tuple:
            np_to_filter = s[1]
            if not np_to_filter:
                for np in sen['en_sen']:
                    if type(np) == tuple:
                        if np[0] == s[0]:
                            np_to_filter = np[1]
            lemmas = filter_nps(np_to_filter, "en")
            en_nps[s[0]] = lemmas
            
    for s in sen['hu_sen']:
        if type(s) == tuple:
            np_to_filter = s[1]
            if not np_to_filter:
                for np in sen['hu_sen']:
                    if type(np) == tuple:
                        if np[0] == s[0]:
                            np_to_filter = np[1]
            lemmas = filter_nps(np_to_filter, "hu")
            hu_nps[s[0]] = lemmas
    
            
    scores = [[] for i in range(len(en_nps))]
    embedding_scores = [[] for i in range(len(en_nps))]
    
    for en_np in en_nps:
        for hu_np in hu_nps:
            l = []
            hu_lower = [s.lower() for s in hu_nps[hu_np]]
            
            #add_morphs = []
            #for low in hu_lower:
            #    if count_vowels(low) > 3:
            #        ms = return_morph(low)
            #        add_morphs += ms
            #for addit in add_morphs:
            #    if addit not in hu_lower:
            #        hu_lower.append(addit)
            
            for word in en_nps[en_np]:
                dic_elements = []
                w = word.strip("-")
                if not dictionary[w]:
                    dic_elements.append(w)
                for el in dictionary[w]:
                     #for i in el.split():
                    #process_el = filter_nps([el], "hu")
                    dic_elements.append(el)
                inter = []
                for en_word in dic_elements:
                    for hu_word in hu_lower:
                        dis = distance(en_word, hu_word)
                        
                        if(len(en_word) > 5 or len(hu_word) > 5):
                            if dis < 3:
                                inter.append(True)
                        else:
                            if dis < 1:
                                inter.append(True)
                                
                l.append(len(inter) > 0)
            listmax = max([hu_lower, en_nps[en_np]], key=len)
            if len(listmax) == 0:
                score = 0
            else:
                score = float(l.count(True)/len(listmax))
            scores[en_np].append(score)
            
            emb_score = get_embedding_score(en_nps[en_np], hu_lower)
            embedding_scores[en_np].append(emb_score)
    return [scores, embedding_scores]

In [38]:
def process(sen):
    scores = compute_scores(sen)
    if scores[0] is None:
        return None
    aligns = []
    for i in range(len(scores[0])):
        for j,k in enumerate(scores[0][i]):
            #if (float(k) > 0.01) or float(scores[1][i][j]) > 0.5:
            if float(scores[1][i][j]) >= 0.5:
            #and float(scores[1][i][j]) > 0.1:
                aligns.append((str(i), str(j)))
    return aligns

In [ ]:
guesses = []
senaligns = {}
for i,sentence in enumerate(sentences):
    print(i)
    gold = sentence['aligns']
    gold_filtered = []
    for goldalign in gold:
        en = re.findall('\d+', goldalign[0] )
        hu = re.findall('\d+', goldalign[1] )
        gold_filtered.append((str(en[0]), str(hu[0])))
    al = process(sentence)
    senaligns[sentence['id']] = al
    if al is not None:        
        for i in al:
            if i in gold_filtered:
                guesses.append(True)
            else:
                guesses.append(False)
        #for g in gold_filtered:
        #    if g not in al:
        #        print(g)
        #        print(sentence['id'])

In [45]:
score = float(guesses.count(True)/len(guesses))
np_len = 0
for sen in sentences:
    np_len += len(sen['aligns'])
print(score)
print(np_len)
print(len(guesses))
recall = (score * len(guesses)) / np_len
f1_score = (2*recall*score)/(recall+score)
print(recall)
print(f1_score)

0.6561523495076941
18848
16961
0.5904605263157895
0.621575581557709


In [ ]:
0.7006302521008403
2506
2856
0.798483639265762
0.7463632972771354

only morph and dict
0.768337505992649
18848
18773
0.7652801358234296
0.7668057733712554

embedding and morph 0,6
0.7508176923463996
18848
19873
0.7916489813242784
0.7706929056584283

embedding and morph 0,6 on stop filtered
0.8114015176081458
16146
15419
0.7748668400842313
0.7927134484397276

embedding+4lang + morph 0,6 0.7417228317016775
18848
20327
0.7999257215619694
0.7697255902999361

In [26]:
import json
from pprint import pprint

with open('hu_ancestors.json') as f:
    hu_ancestors = json.load(f)
with open('en_ancestors.json') as f:
    en_ancestors = json.load(f)

In [27]:
en_ancestors_filtered = {}
for ancestor in en_ancestors:
    anc_list = en_ancestors[ancestor]
    anc = ancestor.strip("-")
    en_ancestors_filtered[anc] = anc_list

In [28]:
en_ancestors = en_ancestors_filtered

In [29]:
import numpy as np

In [30]:
ancestor_count = defaultdict(int)
for ancestor in en_ancestors:
    for a in en_ancestors[ancestor]:
        ancestor_count[a] += 1

In [31]:
sorted_by_value = sorted(ancestor_count.items(), key=lambda kv: kv[1])

In [32]:
en_ancestor_top = sorted_by_value[-500:]

In [33]:
en_ancestor_top = [word[0] for word in en_ancestor_top]

In [34]:
ancestor_count = defaultdict(int)
for ancestor in hu_ancestors:
    for a in hu_ancestors[ancestor]:
        ancestor_count[a] += 1

In [35]:
sorted_by_value = sorted(ancestor_count.items(), key=lambda kv: kv[1])

In [36]:
hu_ancestor_top = sorted_by_value[-500:]

In [37]:
hu_ancestor_top = [word[0] for word in hu_ancestor_top]

In [ ]:
sorted_by_value[-500:]

In [ ]:
guesses_stop = []
senaligns = {}
for i,sentence in enumerate(stop_sentences):
    print(i)
    gold = sentence['aligns']
    gold_filtered = []
    for goldalign in gold:
        en = re.findall('\d+', goldalign[0] )
        hu = re.findall('\d+', goldalign[1] )
        gold_filtered.append((str(en[0]), str(hu[0])))
    al = process(sentence)
    senaligns[sentence['id']] = al
    if al is not None:        
        for i in al:
            if i in gold_filtered:
                guesses_stop.append(True)
            else:
                guesses_stop.append(False)
        #for g in gold_filtered:
        #    if g not in al:
        #        print(g)
        #        print(sentence['id'])

In [164]:
hu_ancestor_top_filtered = []
en_ancestor_top_filtered = []

In [165]:
with open('en_ancestor_top', 'r') as f:
    for item in f:
        en_ancestor_top_filtered.append(item.strip("\n"))
        
with open('hu_ancestor_top', 'r') as f:
    for item in f:
        hu_ancestor_top_filtered.append(item.strip("\n"))